In [65]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [66]:
import os
import sys

# TODO: change this to the path to your homework folder
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = '/content/drive/MyDrive/Junior Year/EECS 487/Group Project'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))
os.chdir(GOOGLE_DRIVE_PATH)

['Project Proposal Brainstorm.gdoc', 'cleaned_headlines.csv', 'vectors.kv', 'vectors.kv.vectors.npy', 'Questions for Prof.gdoc', 'Notes For Progress Report.gdoc', 'BERT.ipynb', 'LSTM.ipynb']


In [67]:
# Sadly we are here as getting TensorFlow on a laptop is a bit of a nightmare
import tensorflow as tf

from tensorflow.keras.datasets import imdb
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences

import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split

In [187]:
# Model configuration
additional_metrics = ['accuracy']
batch_size = 128
embedding_output_dims = 100
loss_function = BinaryCrossentropy()
num_distinct_words = 15000
number_of_epochs = 5
optimizer = Adam()
validation_split = 0.20
verbosity_mode = 1

In [188]:
# Disable eager execution
tf.compat.v1.disable_eager_execution()

In [189]:
# # Load dataset
# (x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_distinct_words)
# print(x_train.shape)
# print(x_test.shape)

In [190]:
data = pd.read_csv("./cleaned_headlines.csv", index_col="Unnamed: 0")
embed = KeyedVectors.load('vectors.kv')
unk_glove_rep = np.mean(embed.vectors, axis=0)
data.head()

,is_sarcastic,headline,article_link,no_stopwords,tokenized,tokenized_no_stopwords
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...,thirtysomething scientists unveil doomsday clo...,thirtysomething scientists unveil doomsday clo...,thirtysomething scientists unveil doomsday clo...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...,dem rep. totally nails congress falling short ...,dem rep. totally nails why congress is falling...,dem rep. totally nails congress falling short ...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...,eat veggies: 9 deliciously different recipes,eat your veggies : 9 deliciously different rec...,eat veggies : 9 deliciously different recipes
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...,inclement weather prevents liar getting work,inclement weather prevents liar from getting t...,inclement weather prevents liar getting work
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...,mother comes pretty close using word 'streamin...,mother comes pretty close to using word 'strea...,mother comes pretty close using word 'streamin...


In [191]:
# For now we'll do tokenized with stopwords
data["split_tokens"] = [headline.split(" ") for headline in data.tokenized]
max_sequence_length = max([len(headline) for headline in data.split_tokens])

In [192]:
# Load up the numeric word encoder
word_index = imdb.get_word_index()

# Compute embeddings
X = []
for headline in data.split_tokens:
  word_lst = []
  for token in headline:
    try:
      temp = word_index[token]
      word_lst.append(temp) if temp < num_distinct_words else word_lst.append(0)
    except:
      word_lst.append(0)
  X.append(word_lst)
X = np.array(X, dtype=object)
print(X)

# Extract the target variable
y = data.is_sarcastic

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=487)

# Pad all sequences
padded_inputs = pad_sequences(X_train, maxlen=max_sequence_length, value = 0.0) # 0.0 because it corresponds with <PAD>
padded_inputs_test = pad_sequences(X_test, maxlen=max_sequence_length, value = 0.0) # 0.0 because it corresponds with <PAD>

[list([0, 3365, 0, 0, 5428, 4, 1150, 1934])
 list([0, 0, 481, 5570, 135, 11789, 6, 1451, 343, 20, 4669, 0, 4343, 14758])
 list([1897, 126, 0, 0, 787, 6919, 272, 0]) ...
 list([1, 88, 304, 5604, 2488, 11, 1266, 382, 36, 3, 14343, 3451])
 list([4596, 0, 2727, 31, 0, 0, 831, 5058])
 list([1243, 0, 11, 21, 3, 1641, 567])]


In [193]:
padded_inputs

array([[    0,     0,     0, ..., 13179,   205,   147],
       [    0,     0,     0, ...,     0,  3447,     0],
       [    0,     0,     0, ...,     0,    15,    22],
       ...,
       [    0,     0,     0, ...,     9,    13,  5371],
       [    0,     0,     0, ...,  1630,    77,    64],
       [    0,     0,     0, ...,     1,  5939,     0]], dtype=int32)

In [194]:
# Define the Keras model
model = Sequential()
model.add(Embedding(num_distinct_words, embedding_output_dims, input_length=max_sequence_length))
model.add(LSTM(10))
model.add(Dense(1, activation='sigmoid'))

In [195]:
# Compile the model
model.compile(optimizer=optimizer, loss=loss_function, metrics=additional_metrics)

In [196]:
# Give a summary
model.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_21 (Embedding)    (None, 166, 100)          1500000   
                                                                 
 lstm_22 (LSTM)              (None, 10)                4440      
                                                                 
 dense_22 (Dense)            (None, 1)                 11        
                                                                 
Total params: 1,504,451
Trainable params: 1,504,451
Non-trainable params: 0
_________________________________________________________________


In [197]:
# Train the model
history = model.fit(padded_inputs, y_train, batch_size=batch_size, epochs=number_of_epochs, verbose=verbosity_mode, validation_split=validation_split)

Train on 18316 samples, validate on 4579 samples
Epoch 1/5
18316/18316 [==============================] - ETA: 0s - loss: 0.5873 - accuracy: 0.6832

/usr/local/lib/python3.9/dist-packages/keras/engine/training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


18316/18316 [==============================] - 21s 1ms/sample - loss: 0.5873 - accuracy: 0.6832 - val_loss: 0.4950 - val_accuracy: 0.7917
Epoch 2/5
18316/18316 [==============================] - 18s 987us/sample - loss: 0.3741 - accuracy: 0.8540 - val_loss: 0.4030 - val_accuracy: 0.8279
Epoch 3/5
18316/18316 [==============================] - 18s 998us/sample - loss: 0.2619 - accuracy: 0.9007 - val_loss: 0.4037 - val_accuracy: 0.8303
Epoch 4/5
18316/18316 [==============================] - 20s 1ms/sample - loss: 0.1990 - accuracy: 0.9290 - val_loss: 0.4342 - val_accuracy: 0.8198
Epoch 5/5
18316/18316 [==============================] - 18s 991us/sample - loss: 0.1580 - accuracy: 0.9451 - val_loss: 0.4665 - val_accuracy: 0.8225


In [198]:
# Test the model after training
test_results = model.evaluate(padded_inputs_test, y_test, verbose=False)
print(f'Test results - Loss: {round(test_results[0], 3)} - Accuracy: {round(100*test_results[1], 4)}%')

Test results - Loss: 0.434 - Accuracy: 83.0887%
